# Example of single instance scraping

In [ ]:
## Data storage and collection dependencies ##
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import requests
from io import BytesIO
import warnings
warnings.filterwarnings("ignore")
import pickle

In [ ]:
url = "https://tasty.co/recipe/garlic-bacon-shrimp-alfredo"
# url = "https://tasty.co/recipe/white-lasagna-soup"

req = requests.get(url)

content = req.text
content

'<!DOCTYPE html><html data-app="tasty_ui" data-env="prod" lang="en" class="enhanced-ux"><head><meta charSet="utf-8"/><meta http-equiv="Content-Type" content="text/html; charset=utf-8"/><meta name="copyright" content="Copyright BuzzFeed, Inc. All rights reserved."/><meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=5, minimum-scale=1, user-scalable=yes"/><meta name="referrer" content="unsafe-url"/><meta name="apple-mobile-web-app-capable" content="yes"/><meta name="apple-mobile-web-app-title" content="Tasty"/><meta name="theme-color" content="#79DCF1"/><meta name="description" content="Here&#x27;s what you need: bacon, shrimp, salt, black pepper, medium yellow onion, medium tomato, garlic, heavy cream, fettuccine, grated parmesan cheese, fresh parlsey"/><meta name="robots" content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1"/><meta property="og:url" content="https://tasty.co/recipe/garlic-bacon-shrimp-alfredo"/><meta prope

In [ ]:
soup = BeautifulSoup(content)

In [ ]:
#Recipe name

raw=soup.findAll('h1', class_ = 'recipe-name extra-bold xs-mb05 md-mb1')
raw

[<h1 class="recipe-name extra-bold xs-mb05 md-mb1">Garlic Shrimp Bacon Alfredo</h1>]

In [ ]:
#Ingredient name

raw=soup.findAll('li', class_ = 'ingredient xs-mb1 xs-mt0')
raw

[<li class="ingredient xs-mb1 xs-mt0">6 slices <!-- -->bacon<!-- -->, chopped</li>,
 <li class="ingredient xs-mb1 xs-mt0">1 lb <!-- -->shrimp<span class="xs-inline-block xs-ml05"> (<!-- -->455 <!-- -->g<!-- -->)</span>, peeled and deveined</li>,
 <li class="ingredient xs-mb1 xs-mt0">1 teaspoon <!-- -->salt</li>,
 <li class="ingredient xs-mb1 xs-mt0">½ teaspoon <!-- -->black pepper</li>,
 <li class="ingredient xs-mb1 xs-mt0">½ <!-- -->medium yellow onion<!-- -->, diced</li>,
 <li class="ingredient xs-mb1 xs-mt0">1 <!-- -->medium tomato<!-- -->, diced</li>,
 <li class="ingredient xs-mb1 xs-mt0">3 cloves <!-- -->garlic<!-- -->, minced</li>,
 <li class="ingredient xs-mb1 xs-mt0">2 cups <!-- -->heavy cream<span class="xs-inline-block xs-ml05"> (<!-- -->480 <!-- -->mL<!-- -->)</span></li>,
 <li class="ingredient xs-mb1 xs-mt0">1 lb <!-- -->fettuccine<span class="xs-inline-block xs-ml05"> (<!-- -->455 <!-- -->g<!-- -->)</span>, cooked</li>,
 <li class="ingredient xs-mb1 xs-mt0">1 cup <!-- -->

In [ ]:
# Nutrition info

raw = soup.findAll('li', class_ = 'list-unstyled xs-mb1')
raw

[<li class="list-unstyled xs-mb1">Calories<span class="bold"> <!-- -->1096</span></li>,
 <li class="list-unstyled xs-mb1">Fat<span class="bold"> <!-- -->63<!-- -->g</span></li>,
 <li class="list-unstyled xs-mb1">Carbs<span class="bold"> <!-- -->98<!-- -->g</span></li>,
 <li class="list-unstyled xs-mb1">Fiber<span class="bold"> <!-- -->4<!-- -->g</span></li>,
 <li class="list-unstyled xs-mb1">Sugar<span class="bold"> <!-- -->9<!-- -->g</span></li>,
 <li class="list-unstyled xs-mb1">Protein<span class="bold"> <!-- -->49<!-- -->g</span></li>,
 <li class="list-unstyled xs-mb1">Calories<span class="bold"> <!-- -->1096</span></li>,
 <li class="list-unstyled xs-mb1">Fat<span class="bold"> <!-- -->63<!-- -->g</span></li>,
 <li class="list-unstyled xs-mb1">Carbs<span class="bold"> <!-- -->98<!-- -->g</span></li>,
 <li class="list-unstyled xs-mb1">Fiber<span class="bold"> <!-- -->4<!-- -->g</span></li>,
 <li class="list-unstyled xs-mb1">Sugar<span class="bold"> <!-- -->9<!-- -->g</span></li>,
 <

## Now we need a way to get the website links for a bunch of recipes

*   Selenium allows us to click the show more button each search query to collect the recipes



## Dependencies

In [ ]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.ZQicaIIo4d/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.ohu9wGLSIz/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.9s9FW8jHg0/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:3 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:6 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:8 http://archive.ubuntu.com/ubuntu focal InRelease
Get:9 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:12 http://deb.debian.org/debian-security buster/updates/main amd64 Packages [608 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:14 http://ppa.launchpa

In [ ]:
from selenium import webdriver

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

## Navigation and collection

### Example runthrough

In [ ]:
from selenium.webdriver.common.by import By
driver = web_driver()

driver.get("https://tasty.co/search?q=chicken+thigh&sort=popular")




In [ ]:
buttons = driver.find_element(By.CLASS_NAME, 'show-more-button')
driver.execute_script("arguments[0].click();", buttons)

page_source = driver.page_source

In [ ]:
soup = BeautifulSoup(page_source, 'lxml')
# raw = soup.findAll('ul', class_ = 'feed__items list-unstyled')
raw = soup.findAll('ul', class_ = 'feed__items list-unstyled')

recipes = []

for i in range(len(raw)):
  for i1 in range(len(raw[i].findAll('a', href = True))):
    recipes.append(raw[i].findAll('a', href = True)[i1].get('href'))

In [ ]:
len(recipes)

180

### Real Collection

In [ ]:
search_terms = ['chicken', 'pasta', 'rice', 'fish', 'vegetables', 'onion', 'healthy', 'kale', 'potato', 'sweet potato', 'steak', 'pork', 'breakfast', 'dinner', 'lunch', 'fried', 'bake', 'soup', 'indian', 'Mexican', 'Chinese', 'American', 'Italian', 
                'cheese', 'sauce', 'beef', 'satay', 'fried', 'egg', 'work+out', 'green onion','tomato', 'carrot', 'broccoli', 'pepper', 'smoothie', 'braised', 'slow-cooker', 'seafood', 'clams', 'oyster', 'meatball', 'salad', 'fruit', 'low+carb', 'keto',
                'nutritional', 'easy', 'tofu', 'vegetarian', 'vegan', 'garlic', 'stuffed', 'lemon', 'herb', 'grilled', 'ground meat', 'meal prep', 'mediterranean', 'avocado', 'glazed', 'chopped', 'pan', 'one-pot', 'lentil', 'baked', 'spinach', 'tuna', 'dip',
                'cauliflower', 'stew','lettuce', 'homemade', 'roasted', 'salsa', 'salmon', 'bowl', 'wraps', 'mushroom', 'creamy', 'protein', 'tortilla', 'basil', 'roasted', 'beans', 'cookie', 'chocolate', 'sugar', 'cream', 'whipped', 'thighs']
sort_terms = ['popular', 'latest']

In [ ]:
recipes = []

for search in search_terms:
  count = 1
  driver = web_driver()
  driver.get("https://tasty.co/search?q={}&sort={}".format(search, sort_terms[0]))
  buttons = driver.find_element(By.CLASS_NAME, 'show-more-button')
  driver.execute_script("arguments[0].click();", buttons)

  page_source = driver.page_source

  soup = BeautifulSoup(page_source, 'lxml')
  raw = soup.findAll('ul', class_ = 'feed__items list-unstyled')

  for i in range(len(raw)):
    for i1 in range(len(raw[i].findAll('a', href = True))):
      recipes.append(raw[i].findAll('a', href = True)[i1].get('href'))


  driver = web_driver()
  driver.get("https://tasty.co/search?q={}&sort={}".format(search, sort_terms[1]))
  buttons = driver.find_element(By.CLASS_NAME, 'show-more-button')
  driver.execute_script("arguments[0].click();", buttons)

  page_source = driver.page_source

  soup = BeautifulSoup(page_source, 'lxml')
  raw = soup.findAll('ul', class_ = 'feed__items list-unstyled')

  for i in range(len(raw)):
    for i1 in range(len(raw[i].findAll('a', href = True))):
      recipes.append(raw[i].findAll('a', href = True)[i1].get('href'))
      print('Recipe name {} saved...'.format(count))
      count += 1

KeyboardInterrupt: ignored

In [ ]:
len(recipes)

460

In [ ]:
with open('recipes_sample.csv', 'w') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerows(recipes)

## Recipe Extraction

In [ ]:
recipes = pd.read_csv('recipes_sample.csv', header = None)

In [ ]:
len(recipes)

460

In [ ]:
recipe_list = list()

for i in range(len(recipes)):
  x = recipes.iloc[i, :]
  word = str()

  for char in x.values:
    if str(char) != 'nan':
      word += char
  
  recipe_list.append(word)

In [ ]:
#for formatting the xxx for first line (had to be same length as the longest for pandas to work)

recipe_list[0] = '/recipe/cheesy-chicken-alfredo-pasta-bake'

In [ ]:
recipe_list[:5]

['/recipe/cheesy-chicken-alfredo-pasta-bake',
 '/recipe/chicken-veggie-stir-fry',
 '/recipe/sweet-sour-chicken',
 '/recipe/3-ingredient-teriyaki-chicken',
 '/recipe/crispy-creamy-chicken-cordon-bleu']

In [ ]:
#Beautiful soup extraction

recipes = dict(recipe_name = ['Recipe Name'], ingredients = ['Ingredients'], nutrition = ['Nutrition'])

count = 0

for rec in recipe_list:
  url = "https://tasty.co{}".format(rec)
  print(count, ':', url)
  count += 1
  req = requests.get(url)
  content = req.text
  soup = BeautifulSoup(content)
  raw_name = str(soup.findAll('h1', class_ = 'recipe-name extra-bold xs-mb05 md-mb1'))[51:-6]
  
  raw_ingredients = soup.findAll('li', class_ = 'ingredient xs-mb1 xs-mt0')
  ingredients = list()
  for ingredient in raw_ingredients:
    ing = str(ingredient)
    ingredients.append(ing[37:-5])

  raw_nutrition = soup.findAll('li', class_ = 'list-unstyled xs-mb1')
  nutrition = list()
  for nutrition_value in raw_nutrition:
    nut = str(nutrition_value)
    nutrition.append(nut[33:-20])

  recipes['recipe_name'].append(raw_name)
  recipes['ingredients'].append(ingredients)
  recipes['nutrition'].append(nutrition)

0 : https://tasty.co/recipe/cheesy-chicken-alfredo-pasta-bake
1 : https://tasty.co/recipe/chicken-veggie-stir-fry
2 : https://tasty.co/recipe/sweet-sour-chicken
3 : https://tasty.co/recipe/3-ingredient-teriyaki-chicken
4 : https://tasty.co/recipe/crispy-creamy-chicken-cordon-bleu
5 : https://tasty.co/recipe/one-pot-chicken-alfredo
6 : https://tasty.co/recipe/original-orange-chicken-by-panda-express
7 : https://tasty.co/recipe/creamy-chicken-penne-pasta
8 : https://tasty.co/recipe/creamy-lemon-chicken
9 : https://tasty.co/recipe/creamy-lemon-butter-chicken
10 : https://tasty.co/recipe/one-pot-chicken-fajita-pasta
11 : https://tasty.co/recipe/cheesy-chicken-and-broccoli-pasta
12 : https://tasty.co/recipe/hasselback-chicken
13 : https://tasty.co/recipe/cozy-chicken-and-dumplings
14 : https://tasty.co/recipe/classic-chicken-noodle-soup
15 : https://tasty.co/recipe/low-carb-mediterranean-chicken-and-tomato-by-kevin-curry
16 : https://tasty.co/recipe/paprika-chicken-rice
17 : https://tasty.c

In [ ]:
len(recipes['recipe_name'][len(recipes['recipe_name']) > 0])

33

# Recipe DB

## Page traversal for cuisines with downloads of recipe website address

Navigate through the cuisine result pages, collect all the links for the different recipes

In [ ]:
## create web driver function to navigate websites

from selenium import webdriver

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200") #will it be faster if i change this?
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
country_list = ['Argentine', 'Australian', 'Austrian', 'Belgian', 'Brazil', 'French', 'Korean']

In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

driver = web_driver()

# print('loading driver...')
driver.get("https://cosylab.iiitd.edu.in/recipedb/")

# print('Searching...')
#From homepage, looks for the search bar for cuisine by country
search = driver.find_element(By.XPATH, "/html/body/div[2]/div[4]/div/div[3]/form/div[1]/div[1]/input")
search.send_keys(country_list[-1])
time.sleep(0.5)
search.send_keys(Keys.ENTER)

# print('Waiting...')
time.sleep(2)
#on new korean page
# page_source = driver.page_source
    #page_source gets converted to beautiful soup
    #soup = BeautifulSoup(page_source 'lxml')

buttons = driver.find_elements(By.CLASS_NAME, 'page-link')

for ele in buttons:
  print(ele.text)

Previous
1
2
3
4
5
...
33
Next


In [ ]:
driver.execute_script("arguments[0].click();", buttons[7]) 

buttons = driver.find_elements(By.CLASS_NAME, 'page-link')
for ele in buttons:
  print(ele.text)

Previous
1
...
29
30
31
32
33
Next


In [ ]:

for ele in buttons:
  print(ele.text)


# for search in search_terms:
#   count = 1
#   driver = web_driver()
#   driver.get("https://tasty.co/search?q={}&sort={}".format(search, sort_terms[0]))
#   buttons = driver.find_element(By.CLASS_NAME, 'show-more-button')
#   driver.execute_script("arguments[0].click();", buttons)

#   page_source = driver.page_source

#   soup = BeautifulSoup(page_source, 'lxml')
#   raw = soup.findAll('ul', class_ = 'feed__items list-unstyled')

#   for i in range(len(raw)):
#     for i1 in range(len(raw[i].findAll('a', href = True))):
#       recipes.append(raw[i].findAll('a', href = True)[i1].get('href'))


#   driver = web_driver()
#   driver.get("https://tasty.co/search?q={}&sort={}".format(search, sort_terms[1]))
#   buttons = driver.find_element(By.CLASS_NAME, 'show-more-button')
#   driver.execute_script("arguments[0].click();", buttons)

#   page_source = driver.page_source

#   soup = BeautifulSoup(page_source, 'lxml')
#   raw = soup.findAll('ul', class_ = 'feed__items list-unstyled')

#   for i in range(len(raw)):
#     for i1 in range(len(raw[i].findAll('a', href = True))):
#       recipes.append(raw[i].findAll('a', href = True)[i1].get('href'))
#       print('Recipe name {} saved...'.format(count))
#       count += 1

## Alternate

In [ ]:
print('loading driver...')
driver.get("https://cosylab.iiitd.edu.in/recipedb/search_subregion/{}".format('Argentine'))

print('Searching...')
search = driver.find_element(By.XPATH, "/html/body/div[2]/div[4]/div/div[3]/form/div[1]/div[1]/input")

#element example with link
<a href="/recipedb/search_recipeInfo/100040" target="_blank">
                        HEIRLOOM TOMATO SALAD WITH NECTARINES, AVOCADO AND ARUGULA
                      </a>

### LINKS ON CUISINE PAGE

#recipe links
/html/body/div[3]/div[1]/div/table/tbody/tr[3]/td[1]/a
/html/body/div[3]/div[1]/div/table/tbody/tr[1]/td[1]/a
/html/body/div[3]/div[1]/div/table/tbody/tr[6]/td[1]/a
/html/body/div[3]/div[1]/div/table/tbody/tr[20]/td[1]/a #20 per page


#on a nother page
/html/body/div[3]/div[1]/div/table/tbody/tr[1]/td[1]/a

#next button or
/html/body/div[3]/div[2]/div/ul/li[9]/a 
<a class="page-link" style="color: blue">Next</a>


##### INSIDE RECIPE PAGE


## Once you are in the recipe page click the ingredients
/html/body/div[2]/div[3]/div/div[1]/ul/li[2]/a

#Nutrients
/html/body/div[2]/div[2]/div[2]/div/table/tbody # full body of text
/html/body/div[2]/div[2]/div[2]/div/table/tbody/tr[1]/td[2] # first nutrition total fats td[2] is the value 

#for each tr in 
/html/body/div[2]/div[3]/div/div[5]/table/tbody

# inside a td in there
/html/body/div[2]/div[3]/div/div[5]/table/tbody/tr[8]/td[1]/a #tr[8] means eighth ingredient in the thing
  #a has the name of the ingredient, as well as the link for more info if we need

# the other tds td[2:] are the other nutritional values
/html/body/div[2]/div[3]/div/div[5]/table/tbody/tr[8]/td[2] # 1
/html/body/div[2]/div[3]/div/div[5]/table/tbody/tr[8]/td[3] # tablespoon

#And then there is a class of round off but nothing really changes in path
/html/body/div[2]/div[3]/div/div[5]/table/tbody/tr[8]/td[5]





print('Waiting...')
time.sleep(0.5)


In [ ]:
soup = BeautifulSoup(search.page_source, 'lxml')

AttributeError: ignored

# Data building for recipe collector

In [ ]:
for search in search_terms:
  count = 1
  driver = web_driver()
  driver.get("https://tasty.co/search?q={}&sort={}".format(search, sort_terms[0]))
  buttons = driver.find_element(By.CLASS_NAME, 'show-more-button')
  driver.execute_script("arguments[0].click();", buttons)

  page_source = driver.page_source

  soup = BeautifulSoup(page_source, 'lxml')
  raw = soup.findAll('ul', class_ = 'feed__items list-unstyled')

  for i in range(len(raw)):
    for i1 in range(len(raw[i].findAll('a', href = True))):
      recipes.append(raw[i].findAll('a', href = True)[i1].get('href'))


  driver = web_driver()
  driver.get("https://tasty.co/search?q={}&sort={}".format(search, sort_terms[1]))
  buttons = driver.find_element(By.CLASS_NAME, 'show-more-button')
  driver.execute_script("arguments[0].click();", buttons)

  page_source = driver.page_source

  soup = BeautifulSoup(page_source, 'lxml')
  raw = soup.findAll('ul', class_ = 'feed__items list-unstyled')

  for i in range(len(raw)):
    for i1 in range(len(raw[i].findAll('a', href = True))):
      recipes.append(raw[i].findAll('a', href = True)[i1].get('href'))
      print('Recipe name {} saved...'.format(count))
      count += 1

In [ ]:
recipes = dict(recipe_name = 'Recipe Name', ingredients = 'Ingredients', nutrition = 'Nutrition', country = 'Country Name')

for rec in recipe_list:
  url = "https://tasty.co{}".format(rec)
  print(count, ':', url)
  count += 1
  req = requests.get(url)
  content = req.text
  soup = BeautifulSoup(content)
  raw_name = str(soup.findAll('h1', class_ = 'recipe-name extra-bold xs-mb05 md-mb1'))[51:-6]
  
  raw_ingredients = soup.findAll('li', class_ = 'ingredient xs-mb1 xs-mt0')
  ingredients = list()
  for ingredient in raw_ingredients:
    ing = str(ingredient)
    ingredients.append(ing[37:-5])

  raw_nutrition = soup.findAll('li', class_ = 'list-unstyled xs-mb1')
  nutrition = list()
  for nutrition_value in raw_nutrition:
    nut = str(nutrition_value)
    nutrition.append(nut[33:-20])

  recipes['recipe_name'].append(raw_name)
  recipes['ingredients'].append(ingredients)
  recipes['nutrition'].append(nutrition)

In [ ]:
buttons = driver.find_element(By.CLASS_NAME, 'show-more-button')
driver.execute_script("arguments[0].click();", buttons)

page_source = driver.page_source

In [ ]:
soup = BeautifulSoup(page_source, 'lxml')
# raw = soup.findAll('ul', class_ = 'feed__items list-unstyled')
raw = soup.findAll('ul', class_ = 'feed__items list-unstyled')

recipes = []

for i in range(len(raw)):
  for i1 in range(len(raw[i].findAll('a', href = True))):
    recipes.append(raw[i].findAll('a', href = True)[i1].get('href'))

## Creating database

Maybe we can build it into MongoDB
From the list of links we have, collect the actual information for each recipe including nutrition and funnel it into a database and dataframe that we can use

#Plan

* Get recipe names and ingredients and nutrition values
* Identify the "unhealthy" and "healthy" items in each recipe
* When recommended to someone, add a healthier option button that gives suggested subsitutions for each recipe

* Visualization that lets people based on their inputted ingredients, see what other ingredients they can have, sort of like a visual tree

* Cuisine classification

* Generative recipes???

* Front end web development for people to interact with
    * Store in MongoDB maybe and build a full MERN stack app or we can just use Stremlit